In [2]:
%run "Analysis.ipynb"
import ipytest
import shutil
ipytest.autoconfig()  # Konfiguriert ipytest automatisch für das Notebook

# Pfad zur Produktionsdatenbank und CSV-Dateien
prod_db_path = "sqldatabase.sqlite"
prod_train_csv = "train.csv"
prod_ideal_csv = "ideal.csv"
prod_test_csv = "test.csv"

# Pfad zur Testdatenbank und Test-CSV-Dateien
test_db_path = "test_database.sqlite"
test_train_csv = "test_train.csv"
test_ideal_csv = "test_ideal.csv"
test_test_csv = "test_test.csv"

# Kopieren der Produktionsdatenbank und CSV-Dateien für Testumgebung
shutil.copy(prod_db_path, test_db_path)
shutil.copy(prod_train_csv, test_train_csv)
shutil.copy(prod_ideal_csv, test_ideal_csv)
shutil.copy(prod_test_csv, test_test_csv)

# Test der Kernfunktionen

# Test für die Verbindung zur Datenbank
def test_connect_to_db():
    """
    Testet die Methode `connect_to_db`
    """
    data_operation = DataOperation(test_db_path)
    data_operation.connect_to_db()
    assert data_operation.conn is not None, "Verbindung zur Datenbank konnte nicht hergestellt werden."

# Test für den Datenimport
def test_import_data():
    """
    Testet die Methode `import_data`
    """
    data_importer = DataImport(test_db_path)
    data_importer.connect_to_db()
    data_importer.import_data(test_train_csv, 'training_data')
    data_importer.import_data(test_ideal_csv, 'ideal_functions')
    data_importer.import_data(test_test_csv, 'test_data')
  
    # Überprüfung, ob Daten importiert wurden
    train_df = pd.read_sql("SELECT * FROM training_data", data_importer.conn)
    ideal_df = pd.read_sql("SELECT * FROM ideal_functions", data_importer.conn)
    test_df = pd.read_sql("SELECT * FROM test_data", data_importer.conn)

    assert not train_df.empty, "Trainingsdaten wurden nicht korrekt importiert."
    assert not ideal_df.empty, "Ideale Funktionen wurden nicht korrekt importiert."
    assert not test_df.empty, "Testdaten wurden nicht korrekt importiert."
    
 # Test für das Finden der besten Passungen
def test_find_best_fits():
    """
    Testet die Methode `find_best_fits`
    """
    analysis = DataAnalysis("test_database.sqlite")
    analysis.connect_to_db()
    analysis.load_data_from_sql()
    analysis.find_best_fits()
    
    assert not analysis.best_fits_df.empty, "DataFrame mit besten Passungen ist leer."
    assert len(analysis.best_fits_df) == 4, "DataFrame mit besten Passungen enthält nicht die erwartete Anzahl an Zeilen."

# Test für die Berechnung der maximalen Abweichungen
def test_calculate_max_deviations():
    """
    Testet die Methode `calculate_max_deviations`
    """
    analysis = DataAnalysis("test_database.sqlite")
    analysis.connect_to_db()
    analysis.load_data_from_sql()
    analysis.find_best_fits()
    analysis.calculate_max_deviations()
    
    assert len(analysis.max_deviations) == len(analysis.best_fits_df), "Anzahl der maximalen Abweichungen stimmt nicht mit der Anzahl der besten Passungen überein."
    for deviation in analysis.max_deviations.values():
        assert deviation >= 0, "Eine maximale Abweichung ist negativ."

# Test für die Validierung der Testdaten
def test_validate_test_data():
    """
    Testet die Methode `validate_test_data`
    """
    analysis = DataAnalysis("test_database.sqlite")
    analysis.connect_to_db()
    analysis.load_data_from_sql()
    analysis.find_best_fits()
    analysis.calculate_max_deviations()
    analysis.validate_test_data()
    
    assert not analysis.valid_test_results_df.empty, "DataFrame mit validierten Testdaten ist leer."
    assert all(analysis.valid_test_results_df["Abweichung"] >= 0), "Es existiert eine negative Abweichung in den validierten Testdaten."   

# Ausführung der Tests
ipytest.run()

DB-Verbindung erfolgreich hergestellt.
SQL-DB erstellt
Daten erfolgreich importiert zu training_data.
Daten erfolgreich importiert zu ideal_functions.
Daten erfolgreich importiert zu test_data.
SQL-Tabellen erstellt
DB-Verbindung erfolgreich hergestellt.
Analyse und Validierung erfolgreich abgeschlossen.
.....                                                                                        [100%]
5 passed in 0.36s


<ExitCode.OK: 0>